In [1]:
import time
from matplotlib import pyplot as plt
import numpy as np
import pyvisa as visa
import pickle
from tqdm import tqdm

In [2]:
np.set_printoptions(threshold=8)

def set_osc_acq(osc):
    # Configuración de la adquisición de curvas
    osc.write('DAT:ENC RPB')
    osc.write('DAT:WID 1')
    osc.write('DAT:STAR 1')
    osc.write('DAT:STOP 2500')
    osc.write('ACQ:MOD SAMP')

def parse_parameters(wfmp_params):
    """Parsea la información de configuración de las escalas y las almacena en un diccionario.
    Para después simplificar el gráfico de la curva."""
    result = {}
    header_def = [('BYT_Nr', int),
              ('BiT_Nr', int),
              ('ENCdg', str),
              ('BN_Fmt', str),
              ('BYT_Or', str)]
    wfrm1_def = [('WFID', str),
             ('NR_PT', int),
             ('PT_FMT', str),
             ('XUNIT', str),
             ('XINCR', float),
             ('PT_Off', int),
             ('YUNIT', str),
             ('YMULT', float),
             ('YOFf', float),
             ('YZEro', float)]

    full_def = header_def + [(f'wfm1:{k}', c) for k, c in wfrm1_def] \
           + [(f'wfm2:{k}', c) for k, c in wfrm1_def] \
           + [(f'wfm3:{k}', c) for k, c in wfrm1_def] \
           + [(f'wfm4:{k}', c) for k, c in wfrm1_def]
    for (key, conversor), param in zip(full_def, wfmp_params.split(';')):
        result[key] = conversor(param)
    return result

def escalar_curva(data, parametros):
    """Devuelve el par de vectores (t, v) a partir de la curva levantada del osciloscopio
    y los parámetros de escala del mismo. El eje temporal está en ms y el vertical en Volts/"""
    n = len(data)
    t = 1e3 * (np.arange(n) - parametros['wfm1:PT_Off']) * parametros['wfm1:XINCR']

    v = (data - parametros['wfm1:YOFf']) * parametros['wfm1:YMULT'] \
                    + parametros['wfm1:YZEro']
    return t, v


# Definimos una función para obtener los datos de uno de los canales del osciloscopio
def medir_canal(osc, channel):
    osc.write(f'DAT:SOU CH{channel}') # Adquirimos del canal 
    # La lectura se puede hacer en modo binario, lo que la hace más eficiente
    conf_parameters = parse_parameters(osc.query('WFMP?'))
    data = osc.query_binary_values('CURV?', datatype='B', container=np.array)
    
    # Escalamos los valores con las unidades correctas
    t, v = escalar_curva(data, conf_parameters)
    return t, v

In [3]:
#%%
# inicializo comunicacion con equipos
rm = visa.ResourceManager()

# lista de dispositivos conectados, para ver las id de los equipos
print(rm.list_resources())

('GPIB0::1::INSTR', 'GPIB0::7::INSTR')


In [4]:
# Abrimos el recurso de VISA, en este caso es un osciloscopio
osc = rm.open_resource('GPIB0::7::INSTR')

print(osc.query("HOR?"))
set_osc_acq(osc)

func = rm.open_resource("GPIB0::1::INSTR")

MAIN;1000;5.0E1;5.0E-5;50;5.0E-5;RUNSAFTER;1.0E-8;1.0065E-6;LOCK;LOCK;0



# Celda para modificar directamente atributos del generador de funciones

In [45]:
# Para enviar una cuadrada
#func.write("FUNC SQUARE")
# Amplitud de la señal
func.write("AMPL 0.1 VP") # En formato "AMP {valor_amplitud} {unidad, ej: VP:VPP, VR:VRMS}"
# Offset de la señal
func.write("OFFS 2.4")
# Frecuencia de la señal 
#func.write("FREQ 10000")

12

In [68]:
# Offset de la señal
offset = -0.25

func.write(f"OFFS {offset}")

12

In [63]:
func.write("AMPL 0 VP")

11

## Barrido en amplitud

In [302]:
# Offset de la señal
offset = 2.4
correccion_posicion = -2*offset
func.write(f"OFFS {offset}")

# Valores de amplitud que tomará la señal
amp_values = np.linspace(0.1,1.7,20)
# Creamos un diccionario para guardar las mediciones que tomamos
dic = {}

for i, amp in tqdm(enumerate(amp_values)):
    # Cambiamos la amplitud de la señal en voltaje pico a pico
    func.write(f"AMPL {amp} VP")
    print(f"La amplitud cambio a {amp} V")
    # Cambiamos el trigger al valor medio de la señal del canal 1, que es una onda cuadrada
    osc.write("TRIGGER:MAIN SetLevel")
    time.sleep(4)
    
    # Usamos los valores de la anterior medición para cambiar la escala y la posición vertical de la siguiente
    if i >= 1:
        # Cambiamos la posicion vertical de los canales
        #for ch in [1,2]:
            #if ch == 1:
        # Cambiamos la escala del canal 1
        #scale = (2*amp/2)-(2*amp/2)%0.005
        #osc.write(f'CH1:SCALE {scale}')
        #osc.write("TRIGGER:MAIN SetLevel")
        # Cambiamos la posición vertical de las señales
        # La posición se selecciona eligiendo los cuadrados del osciloscopio
        osc.write(f"CH1:POSition {correccion_posicion/scale}")
                
            #else:
              #  scale = 0.05 # 50 mV
        
    # Tomamos mediciones de los canales
    t_1, v_1 = medir_canal(osc,1)
    t_2, v_2 = medir_canal(osc,2)
    
    # Evitamos medir tensiones positivas, ya que sabemos que el laser funciona con corr < 0
    if np.mean(v_2) > 0:
        print("Medí positivo")
    #    break
    
    # Guardamos los datos
    dic[f"{amp}V"] = {"t_1":t_1, "V_1":v_1, "t_2":t_2, "V_2": v_2}


0it [00:00, ?it/s]

La amplitud cambio a 0.1 V


1it [00:07,  7.51s/it]

La amplitud cambio a 0.1842105263157895 V


2it [00:15,  7.85s/it]

...

18it [02:16,  7.52s/it]

La amplitud cambio a 1.6157894736842107 V


19it [02:23,  7.51s/it]

La amplitud cambio a 1.7 V


20it [02:31,  7.57s/it]


In [303]:
# Guardamos los datos en un .pickle
pickle.dump(dic, open(f'../mediciones/Barrido en amplitud a {offset} V de offset.pickle', 'wb'))

## Barrido en offset

In [222]:
a = offs_values[-5]
func.write(f"OFFS 2.4")
osc.write("TRIGGER:MAIN SetLevel")
print(a)

2.499999999999999


In [199]:
scale_1 = f'{(2*amp/2)-(2*amp/2)%0.005}'
osc.write(f'CH1:SCALE {scale_1}')
osc.write("TRIGGER:MAIN SetLevel")

23

In [178]:
"En este encontramos que se rompe"
2.9473684210526314 

2.9473684210526314

In [193]:
len(np.arange(-1,3,0.1))

40

In [196]:
# Lista con los valores de offset a barrer
offs_values = np.arange(-1,3,0.1)

# Amplitud de la señal
amp = 1
func.write(f"AMPL {amp} VP") # En formato "AMP {valor_amplitud} {unidad, ej: VP:VPP, VR:VRMS}"
# Cambiamos la escala del canal 1
scale_1 = (2*amp/2)-(2*amp/2)%0.005
osc.write(f'CH1:SCALE {scale_1}')
osc.write("TRIGGER:MAIN SetLevel")

# Diccionario para guardar las mediciones
dic = {}

for i, offset in tqdm(enumerate(offs_values)):
    print("ESTE ES EL OFFSET", offset)
    # Cambiamos el offset de la señal
    func.write(f"OFFS {offset}")
    #print(f"El offset cambio a {offset} V")
    
    # Cambiamos el trigger al valor medio de la señal del canal 1, que es una onda cuadrada
    osc.write("TRIGGER:MAIN SetLevel")
    time.sleep(4)
    
    # Usamos los valores de la anterior medición para cambiar la escala y la posición vertical de la siguiente
    if i >= 1:
        # Cambiamos la posicion vertical de los canales
        for ch in [1,2]:
            if ch == 1:
                scale = scale_1
                # Agregamos una corrección para que el canal 1 quede centrado
                correc_canal = 0.5
            else:
#                 # Cambiamos la escala del canal 2
#                 maximo = max(dic[f"{offs_values[i-1]}V"][f"V_2"])
#                 minimo = min(dic[f"{offs_values[i-1]}V"][f"V_2"])
#                 # Determinamos la amplitud de la cuadrada
#                 amp = maximo-minimo
#                 # Calculamos la escala y la cambiamos
#                 scale = (2*amp/2)-(2*amp/2)%0.005
#                 if scale != 0:
#                     osc.write(f'CH2:SCALE {scale}')
                scale = 0.05 # 50 mV
                # Agregamos una corrección que no afecta al canal 2
                correc_canal = 0
                
                
            
            # Cambiamos la posición vertical de las señales
            offset_anterior = np.mean(dic[f"{offs_values[i-1]}V"][f"V_{ch}"])
            correccion_posicion = -(offset_anterior-offset_anterior%0.005) - correc_canal
            # La posición se selecciona eligiendo los cuadrados del osciloscopio
            osc.write(f"CH{ch}:POSition {correccion_posicion/scale}")
        
        
    # Tomamos mediciones de los canales
    t_1, v_1 = medir_canal(osc,1)
    t_2, v_2 = medir_canal(osc,2)
    
    # Evitamos medir tensiones positivas, ya que sabemos que el 
    if np.mean(v_2) > 0:
        print("Medí positivo")
    #    break
    
    # Guardamos los datos
    dic[f"{offset}V"] = {"t_1":t_1, "V_1":v_1, "t_2":t_2, "V_2": v_2}


0it [00:00, ?it/s]

ESTE ES EL OFFSET -1.0


1it [00:07,  7.58s/it]

ESTE ES EL OFFSET -0.9


2it [00:16,  8.28s/it]

...

ESTE ES EL OFFSET 2.799999999999999


39it [05:06,  7.69s/it]

ESTE ES EL OFFSET 2.899999999999999


40it [05:14,  7.86s/it]

Medí positivo


In [197]:
# Guardamos los datos en un .pickle
pickle.dump(dic, open(f'../mediciones/Barrido fino en offsets a {amp} V de amp.pickle 2', 'wb'))

## Barrido en frecuencias

In [64]:
freq_list = np.linspace(9000, 11000, 15)

for freq in tqdm(freq_list):
    
    func.write(f"FREQ {freq}")
    #print(f"La frecuencia cambió a {freq}")
    time.sleep(10)

print("\n Terminó el barrido")

La frecuencia cambió a 9000.0
La frecuencia cambió a 9142.857142857143
La frecuencia cambió a 9285.714285714286
La frecuencia cambió a 9428.57142857143
La frecuencia cambió a 9571.42857142857
La frecuencia cambió a 9714.285714285714
La frecuencia cambió a 9857.142857142857
La frecuencia cambió a 10000.0
La frecuencia cambió a 10142.857142857143
La frecuencia cambió a 10285.714285714286
La frecuencia cambió a 10428.57142857143
La frecuencia cambió a 10571.428571428572
La frecuencia cambió a 10714.285714285714
La frecuencia cambió a 10857.142857142857
La frecuencia cambió a 11000.0
